In [ ]:
import pandas as pd
df=pd.read_csv("/content/dialogueText.csv")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1038324 entries, 0 to 1038323
Data columns (total 6 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   folder      1038324 non-null  int64 
 1   dialogueID  1038324 non-null  object
 2   date        1038324 non-null  object
 3   from        1038311 non-null  object
 4   to          566035 non-null   object
 5   text        1038235 non-null  object
dtypes: int64(1), object(5)
memory usage: 47.5+ MB


In [ ]:
df.head()

,folder,dialogueID,date,from,to,text
0,3,126125.tsv,2008-04-23T14:55:00.000Z,bad_image,NaN,"Hello folks, please help me a bit with the fol..."
1,3,126125.tsv,2008-04-23T14:56:00.000Z,bad_image,NaN,Did I choose a bad channel? I ask because you ...
2,3,126125.tsv,2008-04-23T14:57:00.000Z,lordleemo,bad_image,the second sentence is better english and we...
3,3,64545.tsv,2009-08-01T06:22:00.000Z,mechtech,NaN,Sock Puppe?t
4,3,64545.tsv,2009-08-01T06:22:00.000Z,mechtech,NaN,WTF?


In [ ]:
import re

def clean_text(text):
    # Handle potential float values
    if isinstance(text, float):
        return str(text)  # Convert floats to strings

    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text

df['cleaned_text'] = df['text'].apply(clean_text)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1038324 entries, 0 to 1038323
Data columns (total 7 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   folder        1038324 non-null  int64 
 1   dialogueID    1038324 non-null  object
 2   date          1038324 non-null  object
 3   from          1038311 non-null  object
 4   to            566035 non-null   object
 5   text          1038235 non-null  object
 6   cleaned_text  1038324 non-null  object
dtypes: int64(1), object(6)
memory usage: 55.5+ MB


In [ ]:
df.head()

,folder,dialogueID,date,from,to,text,cleaned_text
0,3,126125.tsv,2008-04-23T14:55:00.000Z,bad_image,NaN,"Hello folks, please help me a bit with the fol...",hello folks please help me a bit with the foll...
1,3,126125.tsv,2008-04-23T14:56:00.000Z,bad_image,NaN,Did I choose a bad channel? I ask because you ...,did i choose a bad channel i ask because you s...
2,3,126125.tsv,2008-04-23T14:57:00.000Z,lordleemo,bad_image,the second sentence is better english and we...,the second sentence is better english and we...
3,3,64545.tsv,2009-08-01T06:22:00.000Z,mechtech,NaN,Sock Puppe?t,sock puppet
4,3,64545.tsv,2009-08-01T06:22:00.000Z,mechtech,NaN,WTF?,wtf


In [ ]:
# Convert 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Fill null values in 'from', 'to', and 'text' columns with empty strings
df['from'].fillna('', inplace=True)
df['to'].fillna('', inplace=True)
df['text'].fillna('', inplace=True)

# Display the first few rows of the cleaned data
print(df.head())


   folder  dialogueID                      date       from         to  \
0       3  126125.tsv 2008-04-23 14:55:00+00:00  bad_image              
1       3  126125.tsv 2008-04-23 14:56:00+00:00  bad_image              
2       3  126125.tsv 2008-04-23 14:57:00+00:00  lordleemo  bad_image   
3       3   64545.tsv 2009-08-01 06:22:00+00:00   mechtech              
4       3   64545.tsv 2009-08-01 06:22:00+00:00   mechtech              

                                                text  \
0  Hello folks, please help me a bit with the fol...   
1  Did I choose a bad channel? I ask because you ...   
2  the second sentence is better english   and we...   
3                                       Sock Puppe?t   
4                                               WTF?   

                                        cleaned_text  
0  hello folks please help me a bit with the foll...  
1  did i choose a bad channel i ask because you s...  
2  the second sentence is better english   and we...  
3   

In [ ]:
df1 = df.sample(frac=0.25, random_state=1)

In [ ]:
import pandas as pd

# Drop rows with missing values
df1_cleaned_drop = df1.dropna()

# Fill missing values with specific strategies
df1_cleaned_fill = df1.copy()
df1_cleaned_fill['from'] = df1_cleaned_fill['from'].fillna('unknown_sender')
df1_cleaned_fill['to'] = df1_cleaned_fill['to'].fillna('unknown_recipient')
df1_cleaned_fill['text'] = df1_cleaned_fill['text'].fillna('no_text')

# You can choose which strategy to apply
# For example, using the cleaned data with filled values
df1 = df1_cleaned_fill

# If you prefer dropping rows with missing values
# df1 = df1_cleaned_drop

# Summary of the missing data handling
missing_data_summary = df1.isnull().sum()

print(missing_data_summary)

folder          0
dialogueID      0
date            0
from            0
to              0
text            0
cleaned_text    0
dtype: int64


Feature Engineering

Tokenization and lemitization

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

# Download the required resource for part-of-speech tagging
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger') # Download the missing resource

lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize_text(text):
    tokens = word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in tokens]
    return ' '.join(lemmatized_tokens)

# Assuming 'cleaned_text' column exists in your DataFrame
df1['lemmatized_text'] = df1['cleaned_text'].apply(lemmatize_text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


TF-IDF Vectorization

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=5000)
X_df1 = tfidf.fit_transform(df1['lemmatized_text']).toarray()

# Convert to DataFrame
X_df1 = pd.DataFrame(X_df1, columns=tfidf.get_feature_names_out())


Class Distribution

In [ ]:
from sklearn.decomposition import PCA
# Visualizing class separability - PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(X_df1)

plt.figure(figsize=(10, 7))
sns.scatterplot(x=pca_result[:, 0], y=pca_result[:, 1], hue=df1['to'])
plt.title('PCA of TF-IDF Features')
plt.show()

# Visualizing class separability - t-SNE
tsne = TSNE(n_components=2, random_state=42)
tsne_result = tsne.fit_transform(X_df1)

plt.figure(figsize=(10, 7))
sns.scatterplot(x=tsne_result[:, 0], y=tsne_result[:, 1], hue=df1['to'])
plt.title('t-SNE of TF-IDF Features')
plt.show()


NameError: name 'plt' is not defined

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 259581 entries, 1019717 to 740343
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype              
---  ------           --------------   -----              
 0   folder           259581 non-null  int64              
 1   dialogueID       259581 non-null  object             
 2   date             259581 non-null  datetime64[ns, UTC]
 3   from             259581 non-null  object             
 4   to               259581 non-null  object             
 5   text             259581 non-null  object             
 6   cleaned_text     259581 non-null  object             
 7   lemmatized_text  259581 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(1), object(6)
memory usage: 17.8+ MB


In [ ]:
from sklearn.ensemble import RandomForestClassifier # Import the RandomForestClassifier
# Feature importance using RandomForest
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_df1, df1['to'])

# Get feature importances
feature_importances = pd.DataFrame(clf.feature_importances_, index=X_df1.columns, columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances.head(20))


Building the Preprocessing Pipeline